In [1]:
import re
import csv
from tqdm import tqdm
import json
from datetime import datetime
import time
import requests
import os
import sys

sys.path.append("pkg")
sys.path.append(os.path.join("pkg", "preproc"))
from pkg.preproc import base

#### Objectives

1. Identify number of potential speakers (clustering)
    1.1 Use Factors: Readability (F-K), doc_length, n_mentions, n_hashtags, time_of_day??, avg_word_size, avg_num_syllables
2. Attempt to identify Trump (with 'known' data)

In [2]:
from mod import cluster
import numpy as np
from collections import Counter
from pandas import DataFrame
dat = base.Data()
prep = base.Preprocessor()
dat.get_data(load_local=True)

prep.tokenize(data=dat.data)

ids_ = [d.ID for d in prep.docs]

 27%|██▋       | 4753/17433 [02:53<07:28, 28.28it/s]

Length Error: 
4746) Iran humiliated the United States with the capture of our 10 sailors. Horrible pictures and images. We are weak. I will NOT forget!
Length Error: 
4748) Iran humiliated the United States with the capture of our 10 sailors. Horrible pictures and images. We are weak. I will NOT forget!


100%|██████████| 17433/17433 [09:56<00:00, 29.22it/s]


In [5]:
mod0 = cluster(data=prep.arrs, cluster_meth='optics', xi=.00025, 
               metric='euclidean', min_cluster_size=500)

mod0_labs = mod0.labels_
print(np.unique(mod0_labs))
print(dict(Counter(mod0_labs)))

print(len(mod0.labels_))
fields = ["n_hashtags", "n_mentions", "avg_syllables", 
          "avg_word_length", "fk", 'n_sents', 
          "n_ents", "n_uppers", "platform_id",'amplifier',
          'analneg', 'attribadj', 'auxdo', 'bemv',
          'bracket', 'caps', 'cconj', 'cntrstconj',
          'colon', 'comma', 'defart', 'detquan',
          'exclam', 'fstpp', 'fulstop', 'gerund',
          'havemv', 'imperative', 'indefart', 
          'infinitive', 'it', 'mdnec', 
          'mdposs', 'mdpred', 'multiwvb',
          'nomin', 'numdet', 'numnoun', 'objpro',
          'otheradv', 'othrintj', 'othrnoun',
          'othrverb', 'passive', 'past', 'perceptvb',
          'perfect', 'posesprpn', 'possdet', 'predadj', 
          'prep', 'procontract', 'progressive', 
          'proquan', 'provdo', 'prpn', 'prvv', 'pubv', 
          'ques', 'relclausesubgap', 'sinflect', 
          'sndpp', 'stancevb', 'subjpro', 'superlative', 
          'thrdpp', 'timeadv', 'whw', 'initialmention']

df = DataFrame(prep.arrs, index=ids_, columns=fields)
df_simp = DataFrame(index=ids_)
df.insert(len(df.columns), "cluster", mod0_labs)
tod_map = {}
dates, times, tokens = [], [], []
all_tokens = []
for d in prep.docs:
    dates.append(datetime.strptime(d.local_date, "%Y-%m-%d"))
    tm = time.strptime(d.local_time, "%H:%M:%S")
    times.append(tm)
    tokens.append(d.get_tokens_merged(lowercase=True))
    all_tokens.extend(d.get_tokens_merged(lowercase=True))
df.insert(0, "tokens", tokens)
df.insert(0, 'date', dates)
df.insert(0, 'time', times)



Fitting OPTICS(metric='l1', min_cluster_size=500, xi=0.00025)...


/home/magicman/.local/lib/python3.6/site-packages/sklearn/cluster/_optics.py:804: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


Fitted.

Pickling...
Saved.
[-1  0]
{-1: 16766, 0: 551}
17317


KeyError: 'tokens_merged'

In [ ]:
clusts = df['cluster']

outlier_filt = clusts == -1
clust_filt = clusts != -1

df_outliers = df.where(outlier_filt)
df_clusts = df.where(clust_filt)
df_outliers = df_outliers.dropna()
df_clusts = df_clusts.dropna()
grps = df_clusts.groupby(['cluster'])

In [ ]:

from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
import pyLDAvis
import pyLDAvis.gensim


pyLDAvis.enable_notebook(local=True)
model_file = '/tmp/models/lda/base.model'

id2word = Dictionary(tokens)
#print(list(cmn_dict.items()))

def base_model(f, *args, **kwargs):
    if os.path.isfile(f):
        mod = LdaModel.load(f)
    else:
        mod = LdaModel(cmn_corpus, *args, **kwargs)
        mod.save(model_file)
    return mod

def convert_id_to_word(_id, id2word_dict):
    dct = dict(id2word_dict)
    if all([type(dk) == str for dk in dct]):
        dct_type=str
    else:
        dct_type=int
    try:
        w = dct[dct_type.__call__(_id)]
    except KeyError as errmsg:
        raise KeyError(f"invalid id {_id} not found in {dct}")
    else:
        return w
    
for grp in grps:
    n_topics = 20
    grp_tokens = list(grp[1]['tokens'].values)

    grp_corpus = [cmn_dict.doc2bow(text) for text in grp_tokens]
    
    mod = LdaModel(grp_corpus, id2word=dict(cmn_dict.items()), num_topics=n_topics, alpha='auto')
    
    for topic_id in range(n_topics):
        terms = [(convert_id_to_word(x[0], id2word), x[1]) for x in list(mod.get_topic_terms(topic_id))]
        
        

In [ ]:
pre2016_df = df.loc[idx for idx in df.index 
                    if df.loc[idx]['date'] <=  datetime(2017, 1, 20)]

print(pre2016_df)

post2016_df = df.loc[[idx for idx in df.index 
                      if df.loc[idx]['date'] > datetime(2017, 1, 20)]]




